In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
trips19 = pd.read_csv('tripdata_2019-05.csv')
trips20 = pd.read_csv('tripdata_2020-05.csv')

/home/khodor/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# для того, чтобы не засорять оперативу непонятными данными, удалим их на время
del_columns = ['VendorID', 'RatecodeID', 'store_and_fwd_flag', 'extra', 'mta_tax', 'tip_amount', 'improvement_surcharge', 'congestion_surcharge']
trips19 = trips19.drop(columns=del_columns)
trips20 = trips20.drop(columns=del_columns)

In [4]:
max_passengers_count = trips20['passenger_count'].value_counts().idxmax()
print("Max passengers count =", max_passengers_count)
trips20.passenger_count = trips20.passenger_count.fillna(max_passengers_count)

Max passengers count = 1.0


In [5]:
trips19['tpep_pickup_datetime'] = pd.to_datetime(trips19.tpep_pickup_datetime)
trips19['tpep_dropoff_datetime'] = pd.to_datetime(trips19.tpep_dropoff_datetime)

In [6]:
trips20['tpep_pickup_datetime'] = pd.to_datetime(trips20.tpep_pickup_datetime)
trips20['tpep_dropoff_datetime'] = pd.to_datetime(trips20.tpep_dropoff_datetime)

In [7]:
trips19 = trips19[trips19.tpep_pickup_datetime <= trips19.tpep_dropoff_datetime]
trips19 = trips19[trips19.PULocationID <= 263]
trips19 = trips19[trips19.DOLocationID <= 263]

In [8]:
trips20 = trips20[trips20.tpep_pickup_datetime <= trips20.tpep_dropoff_datetime]
trips20 = trips20[trips20.PULocationID <= 263]
trips20 = trips20[trips20.DOLocationID <= 263]

In [9]:
MILES_TO_KM = 1609.34

In [10]:
trips19['trip_pickup_day'] = trips19.tpep_pickup_datetime.dt.day
trips19['week_day'] = trips19.tpep_pickup_datetime.dt.dayofweek
trips19['trip_pickup_hour'] = trips19.tpep_pickup_datetime.dt.hour
trips19['trip_dropoff_hour'] = trips19.tpep_dropoff_datetime.dt.hour
trips19['trip_duration'] = trips19.tpep_dropoff_datetime - trips19.tpep_pickup_datetime
trips19['trip_duration_secs'] = trips19.trip_duration.dt.seconds
trips19['trip_distance'] = trips19['trip_distance'] * MILES_TO_KM
trips19['average_velocity'] = trips19['trip_distance'] / trips19['trip_duration_secs']

In [11]:
trips20['trip_pickup_day'] = trips20.tpep_pickup_datetime.dt.day
trips20['week_day'] = trips20.tpep_pickup_datetime.dt.dayofweek
trips20['trip_pickup_hour'] = trips20.tpep_pickup_datetime.dt.hour
trips20['trip_dropoff_hour'] = trips20.tpep_dropoff_datetime.dt.hour
trips20['trip_duration'] = trips20.tpep_dropoff_datetime - trips20.tpep_pickup_datetime
trips20['trip_duration_secs'] = trips20.trip_duration.dt.seconds
trips20['trip_distance'] = trips20['trip_distance'] * MILES_TO_KM
trips20['average_velocity'] = trips20['trip_distance'] / trips20['trip_duration_secs']

In [12]:
trips19 = trips19[trips19.average_velocity <= 50]
trips19 = trips19[trips19.average_velocity > 1]
trips19 = trips19.drop(trips19[(trips19.trip_distance == 0) & (trips19.total_amount > 0) & (trips19.trip_duration_secs > 0)].index)
trips19 = trips19[trips19.total_amount >= 0]

In [13]:
trips20 = trips20[trips20.average_velocity <= 50]
trips20 = trips20[trips20.average_velocity > 1]
trips20 = trips20.drop(trips20[(trips20.trip_distance == 0) & (trips20.total_amount > 0) & (trips20.trip_duration_secs > 0)].index)
trips20 = trips20[trips20.total_amount >= 0]

In [14]:
trips19['is_holiday'] = [x.date().day == 27 for x in trips19['tpep_pickup_datetime']]
trips20['is_holiday'] = [x.date().day == 25 for x in trips20['tpep_pickup_datetime']]

In [17]:
#c19 = trips19.copy(True)
#c20 = trips20.copy(True)

In [210]:
trips19 = c19.copy(True)
trips20 = c20.copy(True)

In [211]:
#trips19 = trips19[trips19.trip_pickup_hour == val]
#trips20 = trips20[trips20.trip_pickup_hour == val]

### Отметим все выезды на карте.

Импортируем данные о точках в каждой зоне такси.
Заведём "укороченную" версию этой таблицы для мерджа.

In [31]:
taxi_zones = pd.read_csv('taxi_zones.csv')
taxi_zones.columns = taxi_zones.columns.str.strip()
taxi_zones = taxi_zones.drop(columns=['Shape_Leng', 'Shape_Area', 'zone', 'borough']).rename(columns={'X': 'lon', 'Y': 'lat'})

In [32]:
taxi_zones

,lon,lat,PULocationID,DOLocationID
0,-74.176786,40.689516,1,1
1,-73.826126,40.625724,2,2
2,-73.849479,40.865888,3,3
3,-73.977023,40.724152,4,4
4,-74.189930,40.550340,5,5
...,...,...,...,...
258,-73.856351,40.899103,259,259
259,-73.903713,40.746798,260,260
260,-74.012919,40.708976,261,261
261,-73.945830,40.776534,262,262


Добавим в таблицы с данными о поездках координаты посадки и высадки пассажиров. 

In [212]:
trips19 = pd.merge(trips19, taxi_zones, on='PULocationID', how='left')
trips19 = trips19.drop(columns=['DOLocationID_y'])
trips19 = trips19.rename(columns={'DOLocationID_x': 'DOLocationID'})

trips19 = pd.merge(trips19, taxi_zones, on='DOLocationID', how='left')
trips19 = trips19.drop(columns=['PULocationID_y'])
trips19 = trips19.rename(columns={'PULocationID_x': 'PULocationID'})

In [213]:
trips20 = pd.merge(trips20, taxi_zones, on='PULocationID', how='left')
trips20 = trips20.drop(columns=['DOLocationID_y'])
trips20 = trips20.rename(columns={'DOLocationID_x': 'DOLocationID'})

trips20 = pd.merge(trips20, taxi_zones, on='DOLocationID', how='left')
trips20 = trips20.drop(columns=['PULocationID_y'])
trips20 = trips20.rename(columns={'PULocationID_x': 'PULocationID'})

Переименуем для удобства колонки с долготой и широтой и удалим строки, в которых есть значения Nan.

In [214]:
trips19 = trips19.rename(columns={'lon_x': 'lon_pu', 'lat_x': 'lat_pu', 'lon_y': 'lon_do', 'lat_y': 'lat_do'})
trips19 = trips19.dropna(subset=trips19.columns)

trips20 = trips20.rename(columns={'lon_x': 'lon_pu', 'lat_x': 'lat_pu', 'lon_y': 'lon_do', 'lat_y': 'lat_do'})
trips20 = trips20.dropna(subset=trips20.columns)

Для визуализации создадим таблицу вида:

| Отправление/прибытие | id поездки | широта | долгота |
|----------------------|------------|--------|---------|

In [215]:
cols = ['lon_pu', 'lat_pu', 'lon_do', 'lat_do']

In [216]:
trips19_vis = trips19.filter(['lon_pu', 'lat_pu', 'lon_do', 'lat_do']).reset_index()
trips19_vis = trips19_vis.rename(columns={'index': 'id'})

tmp_trips = trips19_vis.loc[trips19_vis.index.repeat(len(cols) / 2), ['id']].reset_index(drop=True)
trips19_vis = tmp_trips.join(pd.DataFrame(trips19_vis[cols].to_numpy().reshape(-1,2), columns=['lon','lat']))

trips19_vis = trips19_vis.reset_index()
trips19_vis['label'] = np.where(trips19_vis['index'] % 2 == 0, 'Origin', 'Destination')
trips19_vis = trips19_vis[['label', 'id', 'lon', 'lat']]

trips19_vis.to_csv('trips19_vis.csv', index=False)

In [217]:
trips20_vis = trips20.filter(['lon_pu', 'lat_pu', 'lon_do', 'lat_do']).reset_index()
trips20_vis = trips20_vis.rename(columns={'index': 'id'})

tmp_trips = trips20_vis.loc[trips20_vis.index.repeat(len(cols) / 2), ['id']].reset_index(drop=True)
trips20_vis = tmp_trips.join(pd.DataFrame(trips20_vis[cols].to_numpy().reshape(-1,2), columns=['lon','lat']))

trips20_vis = trips20_vis.reset_index()
trips20_vis['label'] = np.where(trips20_vis['index'] % 2 == 0, 'Origin', 'Destination')
trips20_vis = trips20_vis[['label', 'id', 'lon', 'lat']]

trips20_vis.to_csv('trips20_vis.csv', index=False)